In [4]:
!pip install pydantic langchain-teddynote langchain_community langchain_huggingface langchain_openai

  Using cached langchain_teddynote-0.3.30-py3-none-any.whl.metadata (708 bytes)
  Using cached langchain_community-0.3.7-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain_huggingface-0.1.2-py3-none-any.whl.metadata (1.3 kB)
  Using cached langchain_openai-0.2.9-py3-none-any.whl.metadata (2.6 kB)
  Using cached kiwipiepy-0.20.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.1 kB)
  Using cached rank_bm25-0.2.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached pinecone_client-5.0.1-py3-none-any.whl.metadata (19 kB)
  Using cached pinecone_text-0.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached olefile-0.47-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached pdf2image-1.17.0-py3-none-any.whl.metadata (6.2 kB)
  Using cached anthropic-0.39.0-py3-none-any.whl.metadata (22 kB)
  Using cached deepl-1.20.0-py3-none-any.whl.metadata (28 kB)
  Using cached feedparser-6.0.11-py3-none-any.whl.metadata (2.4 kB)
  Using cached tavily_python-0.5.0-py3-none-any.

In [ ]:
from google.colab import output

output.clear()

In [5]:
import os

from google.colab import userdata

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "LangChain 실습"
# Langchain API key 입력
os.environ["LANGCHAIN_API_KEY"] = userdata.get('langsmith_api_key')

In [6]:
import zipfile

# 데이터 처리 및 분석
import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm
import seaborn as sns
from matplotlib import pyplot as plt
import datetime

from sklearn.metrics.pairwise import cosine_similarity

# 모델 저장
import joblib

# To ignore all warnings
import warnings
warnings.filterwarnings('ignore')



# 데이터 불러오기

In [7]:
bl_data_sampled=pd.read_csv('bl_sample_data.csv')

display(bl_data_sampled.head())

,HSCD,SUPLY_CO_NAME,PRDT_DC_VAL
0,330499,"MESOESTETIC,. S.L.",BODYSHOCK LOCAL REDUCER 200ML (FACE CREAM)
1,330499,"MESOESTETIC,. S.L.",COSMELAN 2 (FACE CREAM)
2,330499,"MESOESTETIC,. S.L.",ANTI STRESS FACE MASK 100ML (FACE CREAM)
3,330499,KRYOLAN GMBH CHEMISCHE FABRIK,BLOT POWDER
4,330499,KRYOLAN GMBH CHEMISCHE FABRIK,DERMACOLOR CAMOUFLAGE FLUID


In [8]:
bk_data_sampled=pd.read_excel('BK_330499_수정.xlsx')

display(bk_data_sampled.head())

,NO,PID,CORPNAME,PRD_NAME,PRD,URL
0,1,3723376,"CELLVIO COSMETIC CO.,LTD.",Doldori Premium Gold Essence,Our patented mask pack redefines skincare with...,https://buykorea.org/ec/prd/selectGoodsDetail....
1,2,3722918,Brade,Exosome Exo-V Skin Boos,The EXO-V Exosome Skin Booster is used in Kore...,https://buykorea.org/ec/prd/selectGoodsDetail....
2,3,3722956,"BIOVAIKOREA Co.,LTD",Oil-Free Ultra-Moisturizing Lotion with Birch Sap,DETAILS\nBENEFITS\n\n\n• Soothing irritated sk...,https://buykorea.org/ec/prd/selectGoodsDetail....
3,4,3722734,Hit Company,CHARNE - Shiny Brightening Cream,Charne Shiny Brightening Cream\n\nVarious natu...,https://buykorea.org/ec/prd/selectGoodsDetail....
4,5,3717513,"Wooanjoo Co.,Ltd",DEAR OHNEUL Singreen Donkey Toner Pad,Product Features:\n\n－ Hypoallergenic: Dermato...,https://buykorea.org/ec/prd/selectGoodsDetail....


# Langsmith , Chat LLM 설정

In [9]:
from langchain_teddynote import logging

# logging.langsmith("KOTRA_BL_data_matching")

In [11]:
from langchain_community.chat_models import ChatPerplexity
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate

from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


os.environ["PPLX_API_KEY"] = userdata.get('perplexity_api_key')
os.environ["OPENAI_API_KEY"] = userdata.get('KOTRA2_openai_api_key')


In [12]:
# api 사용 rate 제한 걸기 필요

from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter_perplexity = InMemoryRateLimiter(
    requests_per_second=0.7,
    check_every_n_seconds=0.1,  # Wake up every 100 ms to check whether allowed to make a request,
    max_bucket_size=10,  # Controls the maximum burst size.
    )

rate_limiter_openai = InMemoryRateLimiter(
    requests_per_second=5,  # maximum query per day - 10000 (tier 1)
    check_every_n_seconds=0.1,  # Wake up every 100 ms to check whether allowed to make a request,
    max_bucket_size=10,  # Controls the maximum burst size.
    )

<ipython-input-12-1e6674359db1>:5: LangChainBetaWarning: Introduced in 0.2.24. API subject to change.
  rate_limiter_perplexity = InMemoryRateLimiter(


In [13]:
chat_perplexity = ChatPerplexity(temperature=0,  model="llama-3.1-sonar-small-128k-chat", rate_limiter=rate_limiter_perplexity)
chat_openai = ChatOpenAI(temperature=0,  model="gpt-4o-mini", rate_limiter=rate_limiter_openai)

# 1. Perplexity API 이용 결과 (model : llama-3.1-sonar-small-128k-chat)

# buyKOREA 설명 처리 프롬프트문

---------------------

+ 제품 설명 분석
+ 20개의 keyword를 부사, 형용사 없이 나열한 하나의 문장으로 출력
+ 제품의 용도를 잘 설명하는 20개의 한 단어 키워드 추출 (**중복 없이**)
+ 영어로 작성 (토큰 수 줄이기 + llm이 잘 인식하도록)
+ **키워드는 절이 아니라 단어여야 함 (다시 강조 - 절로 추출 시 필요없는 부사, 전치사 다수 포함)**  
+ **제품의 용량 관련된 키워드는 제외**

In [ ]:
prompt_buykorea = ChatPromptTemplate.from_messages([
    ("system", """Analyze product description.
    Make educated guesses based on product purpose/category.
    Extract 20 relevant, single-word keywords, no duplicates.
    Output as single sentence: 'keyword1, keyword2, keyword3,...'.
    Exclude adjectives/adverbs. For phrases, select key word only.
    Exclude keywords representing capacity or volume."""),
    ("human", "{input}")
])

chain_buykorea = prompt_buykorea | chat_perplexity

In [ ]:
buykorea_preprocessed_dict_perplexity={}

for index, row in bk_data_sampled.iterrows():
  response_buykorea = chain_buykorea.invoke({"input": row['PRD']})
  print(response_buykorea.content)
  buykorea_preprocessed_dict_perplexity[row['PRD_NAME']]=response_buykorea.content

mask, skincare, convenience, hygiene, innovation, design, application, package, user experience, practicality, elegance, uniqueness, export, clinics, department stores, online shopping, effectiveness, luxury, manufacturing, quality control, essence formulation, mask sheet, adhesion, comfort, hydration, brightening, revitalization, nourishment, visible benefits, indulgence.
exosome, skin, booster, regeneration, elasticity, wrinkle, moisture, growth, factors, anti-aging, dermatology, aesthetic, clinics, home care, needles, scalp, hair care, stem cell, nutrients, hair roots.
Soothing, moisturizing, hydrating, penetrating, fresh, soft, silky, oily, combination, skin-friendly, natural, vitamins, antioxidants, irritated, calming, treating, weightless, subtle, fragrance-free, tea tree, fast absorption, gentle application, daily use.
whitening, wrinkle improvement, elasticity, antioxidant, radiance, skin care, natural ingredients, firmness, vitality, energy, hydration, glycerin, niacinamide, s

In [ ]:
buykorea_preprocessed_dict_perplexity

{'Doldori Premium Gold Essence': 'mask, skincare, convenience, hygiene, innovation, design, application, package, user experience, practicality, elegance, uniqueness, export, clinics, department stores, online shopping, effectiveness, luxury, manufacturing, quality control, essence formulation, mask sheet, adhesion, comfort, hydration, brightening, revitalization, nourishment, visible benefits, indulgence.',
 'Exosome Exo-V Skin Boos': 'exosome, skin, booster, regeneration, elasticity, wrinkle, moisture, growth, factors, anti-aging, dermatology, aesthetic, clinics, home care, needles, scalp, hair care, stem cell, nutrients, hair roots.',
 'Oil-Free Ultra-Moisturizing Lotion with Birch Sap': 'Soothing, moisturizing, hydrating, penetrating, fresh, soft, silky, oily, combination, skin-friendly, natural, vitamins, antioxidants, irritated, calming, treating, weightless, subtle, fragrance-free, tea tree, fast absorption, gentle application, daily use.',
 'CHARNE - Shiny Brightening Cream': '

In [ ]:
buykorea_preprocessed_perplexity=pd.DataFrame.from_dict(buykorea_preprocessed_dict_perplexity, orient='index', columns=['buykorea_preprocessed'])
buykorea_preprocessed_perplexity.reset_index(inplace=True)
buykorea_preprocessed_perplexity.rename(columns={'index':'product_name'}, inplace=True)
buykorea_preprocessed_perplexity.to_csv('buykorea_preprocessed_perplextiy.csv', index=False)

display(buykorea_preprocessed_perplexity.head())

,product_name,buykorea_preprocessed
0,Doldori Premium Gold Essence,"mask, skincare, convenience, hygiene, innovati..."
1,Exosome Exo-V Skin Boos,"exosome, skin, booster, regeneration, elastici..."
2,Oil-Free Ultra-Moisturizing Lotion with Birch Sap,"Soothing, moisturizing, hydrating, penetrating..."
3,CHARNE - Shiny Brightening Cream,"whitening, wrinkle improvement, elasticity, an..."
4,DEAR OHNEUL Singreen Donkey Toner Pad,"pore, skin, hydration, cleansing, exfoliate, t..."


# BL 데이터 이름 처리 프롬프트문

----------------------------

+ **제품 이름에 기반해, 제품 분류와 용도를 설명**
+ **BL 데이터에 존재하는 HSCODE 정보 입력**
+ 20개의 keyword를 부사, 형용사 없이 나열한 하나의 문장으로 출력
+ 제품의 용도를 잘 설명하는 20개의 한 단어 키워드 추출 (**중복 없이**)
+ 영어로 작성 (토큰 수 줄이기 + llm이 잘 인식하도록)
+ **키워드는 절이 아니라 단어여야 함 (다시 강조 - 절로 추출 시 필요없는 부사, 전치사 다수 포함)**  
+ **제품의 용량 관련된 키워드는 제외**

In [ ]:
hscode=330499

prompt_bldata = ChatPromptTemplate.from_messages([
    ("system", f"""Describe product category and purpose based on input.
     HS code: {hscode}.
   Extract 20 relevant, single-word keywords, no duplicates.
    Output only one sentence: 'keyword1, keyword2, keyword3,...'.
     Exclude adjectives/adverbs. For phrases, select key word only.
     Exclude keywords representing capacity or volume."""),
    ("human", "{input}")
])

chain_bldata = prompt_bldata | chat_perplexity

# 같은 330499 내의 유사 품목 제품명
+ (순서대로 세팅 스프레이, 파운데이션, 컨실러)

# 키워드 추출 실험시 사용

P+P FIX+ COCONUT 100ML/3.4FLOZ - (100.00ml Each) - MXJJ010000

STUDIO TECH-NC25 10GM/.35OZ - (10.00 Each) - M1K2120000

STUDIO FINISH CONC-NC30 7GM/.24OZ - (7.00 g Each) - M450190000

In [ ]:
# response_bldata = chain_bldata.invoke({"input": 'BODYSHOCK LOCAL REDUCER 200ML (FACE CREAM) '})
# print(response_bldata.content)

In [ ]:
bldata_preprocessed_dict_perplexity={}

for index, row in tqdm(bl_data_sampled.iterrows()):
  response_bldata = chain_bldata.invoke({"input": row['PRDT_DC_VAL']})
  if index%100==0:
    print(f"Product Name : {row['PRDT_DC_VAL']} \n")
    print(response_bldata.content)
  bldata_preprocessed_dict_perplexity[row['PRDT_DC_VAL']]=response_bldata.content

1it [00:02,  2.08s/it]

Product Name : BODYSHOCK LOCAL REDUCER 200ML (FACE CREAM) 

Body shock, face cream, reducer, local, 200ml.


101it [02:26,  1.52s/it]

Product Name : AVEENO BABY DERMEXA MOISTURING CREAM 206G(1 PAC X 12 PCS) 

Skin care, moisturizing, cream, baby, dermexa, aveeno, soothing, hydration, protection, sensitive, gentle, nourishing, calming, barrier, repair, relief, soothing, hydration, protection.


201it [04:48,  1.41s/it]

...

Product Name : MASKARA (UNBRANDED) 

Maskara, Mascara, Makeup, Cosmetics, Beauty, Product, Unbranded, Eyelash, Enhancement, Application, Base, Coat, Color, Formula, Application, Tool, Product.


701it [20:11,  1.14s/it]

Product Name : FE ALOVERA JUICE 100ML (RETURN TO ORIGIN) (EXP.VIDE S/BILL NO.9172066 & DT-24/12/19) 

aloe, juice, return, origin, bill, date


787it [23:14,  1.77s/it]


AttributeError: 'str' object has no attribute 'choices'

In [ ]:
bldata_preprocessed_perplexity=pd.DataFrame.from_dict(bldata_preprocessed_dict_perplexity, orient='index', columns=['bldata_preprocessed'])
bldata_preprocessed_perplexity.reset_index(inplace=True)
bldata_preprocessed_perplexity.rename(columns={'index':'product_name'}, inplace=True)
bldata_preprocessed_perplexity.to_csv('bldata_preprocessed_perplextiy.csv', index=False)

display(bldata_preprocessed_perplexity.head())

,product_name,bldata_preprocessed
0,BODYSHOCK LOCAL REDUCER 200ML (FACE CREAM),"Body shock, face cream, reducer, local, 200ml."
1,COSMELAN 2 (FACE CREAM),"Skin, cream, face, melanin, cosmetic, treatmen..."
2,ANTI STRESS FACE MASK 100ML (FACE CREAM),"Relaxation, skincare, face, mask, cream, stres..."
3,BLOT POWDER,"Blotting powder, cosmetic, makeup, powder, bea..."
4,DERMACOLOR CAMOUFLAGE FLUID,"Makeup, Camouflage, Fluid, Dermacolor, Cosmeti..."


In [ ]:
bldata_preprocessed_perplexity

,product_name,bldata_preprocessed
0,BODYSHOCK LOCAL REDUCER 200ML (FACE CREAM),"Body shock, face cream, reducer, local, 200ml."
1,COSMELAN 2 (FACE CREAM),"Skin, cream, face, melanin, cosmetic, treatmen..."
2,ANTI STRESS FACE MASK 100ML (FACE CREAM),"Relaxation, skincare, face, mask, cream, stres..."
3,BLOT POWDER,"Blotting powder, cosmetic, makeup, powder, bea..."
4,DERMACOLOR CAMOUFLAGE FLUID,"Makeup, Camouflage, Fluid, Dermacolor, Cosmeti..."
...,...,...
751,COSMETICS -ACTIPHYTE GREEN TEA GL,"Cosmetics, Green Tea, Actiphyte, Extract, Anti..."
752,COSMETICS ACTIPHYTE LAVENDER GL80NP,"Actiphyte, Lavender, Cosmetics, Gl80np, Fragra..."
753,19885 E EXPERT DROPS (3) - COSMETIC PRODUCT (N...,"cosmetics, skincare, beauty, products, drops, ..."
754,14560 N DC TRANSISION - COSMETIC PRODUCT (NOT ...,"cosmetic, product, transition, inhouse"


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


# 2. OpenAI API 이용 결과 (model : chatgpt 4o-mini)

# buyKOREA 설명 처리 프롬프트문

---------------------

+ 제품 설명 분석
+ 20개의 keyword를 부사, 형용사 없이 나열한 하나의 문장으로 출력
+ 제품의 용도를 잘 설명하는 20개의 한 단어 키워드 추출 (**중복 없이**)
+ 영어로 작성 (토큰 수 줄이기 + llm이 잘 인식하도록)
+ **키워드는 절이 아니라 단어여야 함 (다시 강조 - 절로 추출 시 필요없는 부사, 전치사 다수 포함)**  
+ **제품의 용량 관련된 키워드는 제외**

In [33]:
class DescriptionSummary(BaseModel):
    type : str = Field(description="product type")
    purpose : str = Field(description="product purpose")
    brand : str = Field(description='product brand')
    summary : str = Field(description="summary of description")

parser= PydanticOutputParser(pydantic_object=DescriptionSummary)

type_prompt_buykorea = PromptTemplate.from_template(
    """
    Make a summary that focuses on the product's purpose/type.
    Strictly exclude adjectives, adverbs, prepositions, and any information about capacity or volume.
    Ensure the summary is concise and only includes essential nouns and verbs.

    PRODUCT DESCRIPTION: {input}
    FORMAT: {format}
    """
)

type_prompt_buykorea= type_prompt_buykorea.partial(format=parser.get_format_instructions())

chain_buykorea = type_prompt_buykorea | chat_openai | parser

In [ ]:
# prompt_buykorea = ChatPromptTemplate.from_messages([
#     ("system", """Analyze product description.
#     Make educated guesses based on product purpose/category.
#     Extract 20 relevant, single-word keywords, no duplicates.
#     Output as single sentence: 'keyword1, keyword2, keyword3,...'.
#     Exclude adjectives/adverbs. For phrases, select key word only.
#     Exclude keywords representing capacity or volume."""),
#     ("human", "{input}")
# ])
#
# chain_buykorea = prompt_buykorea | chat_openai

In [35]:
buykorea_preprocessed_dict_openai={}

for index, row in bk_data_sampled.iterrows():
  response_buykorea = chain_buykorea.invoke({"input": row['PRD']})
  print(response_buykorea)
  buykorea_preprocessed_dict_openai[row['PRD_NAME']]=response_buykorea

type='mask pack' purpose='skincare treatment' brand='not specified' summary='mask pack offers new application method, enhances user experience, ensures quality through direct manufacturing, contains essence for hydration and revitalization, designed for comfort and absorption'
type='skin booster' purpose='improves skin regeneration, elasticity, wrinkle reduction, moisture retention' brand='EXO-V' summary='used in clinics, home care, enhances scalp nutrients, strengthens hair roots'
type='lotion' purpose='moisturizing skin' brand='not specified' summary='soothe skin, restore moisture, penetrate skin, calm irritation'
type='cream' purpose='whitening, wrinkle improvement' brand='Charne' summary='Natural ingredients support skin health, nutrition, vitality, firmness, elasticity.'
type='skincare pads' purpose='exfoliate, cleanse, hydrate, control sebum, reduce pores' brand='not specified' summary='Hypoallergenic pads with dual-sided design for cleansing and hydration. Clinically proven to r

In [40]:
print(buykorea_preprocessed_dict_openai['Doldori Premium Gold Essence'].type)

mask pack


In [41]:
buykorea_preprocessed_dict_openai

{'Doldori Premium Gold Essence': DescriptionSummary(type='mask pack', purpose='skincare treatment', brand='not specified', summary='mask pack offers new application method, enhances user experience, ensures quality through direct manufacturing, contains essence for hydration and revitalization, designed for comfort and absorption'),
 'Exosome Exo-V Skin Boos': DescriptionSummary(type='skin booster', purpose='improves skin regeneration, elasticity, wrinkle reduction, moisture retention', brand='EXO-V', summary='used in clinics, home care, enhances scalp nutrients, strengthens hair roots'),
 'Oil-Free Ultra-Moisturizing Lotion with Birch Sap': DescriptionSummary(type='lotion', purpose='moisturizing skin', brand='not specified', summary='soothe skin, restore moisture, penetrate skin, calm irritation'),
 'CHARNE - Shiny Brightening Cream': DescriptionSummary(type='cream', purpose='whitening, wrinkle improvement', brand='Charne', summary='Natural ingredients support skin health, nutrition, 

In [ ]:
buykorea_preprocessed_openai=pd.DataFrame.from_dict(buykorea_preprocessed_dict_openai, orient='index', columns=['buykorea_preprocessed'])
buykorea_preprocessed_openai.reset_index(inplace=True)
buykorea_preprocessed_openai.rename(columns={'index':'product_name'}, inplace=True)
buykorea_preprocessed_openai.to_csv('buykorea_preprocessed_openai.csv', index=False)

display(buykorea_preprocessed_openai.head())

,product_name,buykorea_preprocessed
0,Doldori Premium Gold Essence,"mask, pack, skincare, design, application, pac..."
1,Exosome Exo-V Skin Boos,"exosome, growth, factors, skin, regeneration, ..."
2,Oil-Free Ultra-Moisturizing Lotion with Birch Sap,"soothing, skin, moisture, hydration, penetrate..."
3,CHARNE - Shiny Brightening Cream,"cream, skin, ingredients, vitality, energy, nu..."
4,DEAR OHNEUL Singreen Donkey Toner Pad,"hypoallergenic, pore, design, dual, cleansing,..."


# BL 데이터 이름 처리 프롬프트문

----------------------------

+ **제품 이름에 기반해, 제품 분류와 용도를 설명**
+ **BL 데이터에 존재하는 HSCODE 정보 입력**
+ 20개의 keyword를 부사, 형용사 없이 나열한 하나의 문장으로 출력
+ 제품의 용도를 잘 설명하는 20개의 한 단어 키워드 추출 (**중복 없이**)
+ 영어로 작성 (토큰 수 줄이기 + llm이 잘 인식하도록)
+ **키워드는 절이 아니라 단어여야 함 (다시 강조 - 절로 추출 시 필요없는 부사, 전치사 다수 포함)**  
+ **제품의 용량 관련된 키워드는 제외**

, including product type and purpose.

In [71]:
hscode=330499

description_prompt_bldata = PromptTemplate.from_template(
    """
    Generate product description based on hscode and product name.
    Ensure the description is concise and only includes essential nouns and verbs.

    PRODUCT NAME: {input}
    HSCODE : {hscode}
    FORMAT : {format}
    """
)

description_prompt_bldata= description_prompt_bldata.partial(format=parser.get_format_instructions())
description_prompt_bldata= description_prompt_bldata.partial(hscode=hscode)

chain_bldata = description_prompt_bldata | chat_openai | parser

In [72]:
bldata_preprocessed_dict_openai={}

for index, row in tqdm(bl_data_sampled.iterrows()):
  response_bldata = chain_bldata.invoke({"input": row['PRDT_DC_VAL']})
  if index%100==0:
    print(f"Product Name : {row['PRDT_DC_VAL']} \n")
    print(response_bldata, '\n')
  bldata_preprocessed_dict_openai[row['PRDT_DC_VAL']]=response_bldata
  if index==10:
    break

1it [00:01,  1.28s/it]

Product Name : BODYSHOCK LOCAL REDUCER 200ML (FACE CREAM) 

type='face cream' purpose='local reduction' brand='Bodyshock' summary='200ml cream for targeted facial contouring and skin tightening.' 



10it [00:19,  1.99s/it]


In [73]:
bldata_preprocessed_dict_openai

{'BODYSHOCK LOCAL REDUCER 200ML (FACE CREAM)': DescriptionSummary(type='face cream', purpose='local reduction', brand='Bodyshock', summary='200ml cream for targeted facial contouring and skin tightening.'),
 'COSMELAN 2 (FACE CREAM)': DescriptionSummary(type='face cream', purpose='reduce pigmentation', brand='Cosmelan', summary='Cosmelan 2 is a face cream designed to diminish dark spots and even skin tone.'),
 'ANTI STRESS FACE MASK 100ML (FACE CREAM)': DescriptionSummary(type='face cream', purpose='relieve stress and soothe skin', brand='N/A', summary='Anti-stress face mask hydrates and calms skin, promoting relaxation.'),
 'BLOT POWDER': DescriptionSummary(type='Cosmetic Powder', purpose='Absorb excess oil and reduce shine', brand='Generic', summary='Blot powder controls oil and enhances makeup longevity.'),
 'DERMACOLOR CAMOUFLAGE FLUID': DescriptionSummary(type='cosmetic', purpose='camouflage skin imperfections', brand='Dermacolor', summary='Dermacolor Camouflage Fluid provides eff

In [ ]:
bldata_preprocessed_openai=pd.DataFrame.from_dict(bldata_preprocessed_dict_openai, orient='index', columns=['bldata_preprocessed'])
bldata_preprocessed_openai.reset_index(inplace=True)
bldata_preprocessed_openai.rename(columns={'index':'product_name'}, inplace=True)
bldata_preprocessed_openai.to_csv('bldata_preprocessed_openai.csv', index=False)

display(bldata_preprocessed_openai.head())

,product_name,bldata_preprocessed
0,BODYSHOCK LOCAL REDUCER 200ML (FACE CREAM),"cream, body, skincare, beauty, cosmetic, treat..."
1,COSMELAN 2 (FACE CREAM),"cosmetics, cream, face, skincare, treatment, m..."
2,ANTI STRESS FACE MASK 100ML (FACE CREAM),"mask, cream, face, anti, stress, skincare, bea..."
3,BLOT POWDER,"blot, powder, cosmetics, makeup, beauty, appli..."
4,DERMACOLOR CAMOUFLAGE FLUID,"cosmetics, makeup, camouflage, fluid, skin, co..."


# 수출한 기업은 다르지만, 수출한 품목의 이름은 같은 경우가 있어 불일치 발생
(중복 제거됨)

In [ ]:
# 같은 품목 중 다른 기업이 수출한 이력이 있는 경우, 다른 수출건으로 집계됨

bl_data_sampled.shape

(2932, 3)

In [ ]:
# 제품 이름에 기반해, 제품 수입 확률이 높은 해외 바이어 매칭이므로
# 중복되는 제품 이름은 하나의 키워드 및 임베딩으로 통일

len(bldata_preprocessed_dict_openai)

2801

# 임베딩

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

# 오픈소스 임베딩 모델 다운로드
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def text_embedding(text):
  embedding_text=embeddings.embed_query(text)
  return np.array(embedding_text).reshape(1,-1)

In [ ]:
tqdm.pandas()

# 임베딩할 텍스트

buykorea_preprocessed_openai=pd.read_csv('./buykorea_preprocessed_openai.csv')
bldata_preprocessed_openai=pd.read_csv('./bldata_preprocessed_openai.csv')

# 키워드로 구성된 문장을 임베딩
buykorea_preprocessed_openai['buykorea_embedding']=buykorea_preprocessed_openai['buykorea_preprocessed'].progress_apply(text_embedding)
bldata_preprocessed_openai['bldata_embedding']=bldata_preprocessed_openai['bldata_preprocessed'].progress_apply(text_embedding)

bldata_sample_len=bldata_preprocessed_openai.shape[0]

100%|██████████| 2801/2801 [11:31<00:00,  4.05it/s]


In [ ]:
openai_similarity_df=pd.DataFrame(columns=['buykorea_product_name', 'buykorea_keyword', 'bldata_product_name', 'bldata_keyword', 'cosine_similarity'])

for index, row in buykorea_preprocessed_openai.iterrows():

  # 각 buykorea 상품 (10개) 에,
  # 2800개의 수출 기록 존재하는 제품 이름과의 유사도 계산
  # buykorea 상품별로 2800개의 유사도가 계산되어 저장된 데이터프레임 생성

  similarity_df=pd.DataFrame({'buykorea_product_name':[row['product_name']]*bldata_sample_len,
                               'buykorea_keyword': [row['buykorea_preprocessed']]*bldata_sample_len,
                               'bldata_product_name' : ["None"]*bldata_sample_len,
                               'bldata_keyword' : ["None"]*bldata_sample_len,
                               'cosine_similarity' : [0]*bldata_sample_len})

  buykorea_embedding=row['buykorea_embedding']
  row_count=0

  for index, row_bl in tqdm(bldata_preprocessed_openai.iterrows()):

    bldata_embedding=row_bl['bldata_embedding']

    similarity=cosine_similarity(buykorea_embedding, bldata_embedding)
    similarity_df.loc[row_count, 'bldata_product_name']=row_bl['product_name']
    similarity_df.loc[row_count, 'bldata_keyword']=row_bl['bldata_preprocessed']
    similarity_df.loc[row_count, 'cosine_similarity']=similarity

    row_count+=1

  # buykorea 제품 유사도 데이터프레임을 하나의 데이터프레임으로 병합
  openai_similarity_df=pd.concat([openai_similarity_df, similarity_df], axis=0)

display(openai_similarity_df.head())


2801it [00:05, 513.68it/s]
2801it [00:10, 268.78it/s]
2801it [00:05, 485.07it/s]
2801it [00:05, 540.27it/s]
2801it [00:05, 540.83it/s]
2801it [00:04, 605.11it/s]
2801it [00:05, 492.85it/s]
2801it [00:04, 583.80it/s]
2801it [00:04, 585.26it/s]
2801it [00:05, 484.61it/s]


,buykorea_product_name,buykorea_keyword,bldata_product_name,bldata_keyword,cosine_similarity
0,Doldori Premium Gold Essence,"mask, pack, skincare, design, application, pac...",BODYSHOCK LOCAL REDUCER 200ML (FACE CREAM),"cream, body, skincare, beauty, cosmetic, treat...",0.817617
1,Doldori Premium Gold Essence,"mask, pack, skincare, design, application, pac...",COSMELAN 2 (FACE CREAM),"cosmetics, cream, face, skincare, treatment, m...",0.821567
2,Doldori Premium Gold Essence,"mask, pack, skincare, design, application, pac...",ANTI STRESS FACE MASK 100ML (FACE CREAM),"mask, cream, face, anti, stress, skincare, bea...",0.797865
3,Doldori Premium Gold Essence,"mask, pack, skincare, design, application, pac...",BLOT POWDER,"blot, powder, cosmetics, makeup, beauty, appli...",0.757623
4,Doldori Premium Gold Essence,"mask, pack, skincare, design, application, pac...",DERMACOLOR CAMOUFLAGE FLUID,"cosmetics, makeup, camouflage, fluid, skin, co...",0.756394


In [ ]:
openai_similarity_df_sorted= openai_similarity_df.groupby(['buykorea_product_name'], group_keys=False, sort=False).apply(lambda x: x.sort_values('cosine_similarity', ascending=False))
openai_similarity_df_sorted.reset_index(drop=True, inplace=True)
display(openai_similarity_df_sorted.head())

,buykorea_product_name,buykorea_keyword,bldata_product_name,bldata_keyword,cosine_similarity
0,Doldori Premium Gold Essence,"mask, pack, skincare, design, application, pac...",RESUR FACING MOISTURE MASK (COSMETIC) (FOR PER...,"mask, cosmetic, moisture, resurfacing, skincar...",0.899548
1,Doldori Premium Gold Essence,"mask, pack, skincare, design, application, pac...",CITRA GLOW RECIPE JUICY SHEET MASK (COSMETIC S...,"mask, cosmetic, sample, testing, skincare, bea...",0.891938
2,Doldori Premium Gold Essence,"mask, pack, skincare, design, application, pac...",SS0000016020SS APC MASK 13G TUBE,"mask, tube, skin, care, cosmetic, treatment, h...",0.891609
3,Doldori Premium Gold Essence,"mask, pack, skincare, design, application, pac...",WHITENING MASK 18KG-BAG,"mask, whitening, skin, beauty, treatment, care...",0.883059
4,Doldori Premium Gold Essence,"mask, pack, skincare, design, application, pac...",(PCPMACAACYQ-I) DERMAFIQUE BIO CELLULOSE TONE ...,"mask, skincare, beauty, treatment, cellulose, ...",0.881926


In [ ]:
import pandas as pd

# 각 buykorea_product_name 별로 cosine_similarity가 높은 10개, 낮은 10개를 'Highest'로, 나머지는 'Lowest'으로 표시하는 함수
def mark_top_bottom_10(df):
    # 상위 10개와 하위 10개는 'Highest', 나머지는 'Lowest'
    df['test_row'] = ['Highest'] * 10 + ['N'] * (len(df) - 20) + ['Lowest'] * 10
    return df

# 각 buykorea_product_name별로 그룹화하여 처리
# Apply the function and then reset the index to align with the original DataFrame
openai_similarity_df_sorted['test_row'] = openai_similarity_df_sorted.groupby('buykorea_product_name', group_keys=False).apply(mark_top_bottom_10).reset_index(drop=True)['test_row']


# 결과 확인
display(openai_similarity_df_sorted)

,buykorea_product_name,buykorea_keyword,bldata_product_name,bldata_keyword,cosine_similarity,test_row
0,Doldori Premium Gold Essence,"mask, pack, skincare, design, application, pac...",RESUR FACING MOISTURE MASK (COSMETIC) (FOR PER...,"mask, cosmetic, moisture, resurfacing, skincar...",0.899548,Highest
1,Doldori Premium Gold Essence,"mask, pack, skincare, design, application, pac...",CITRA GLOW RECIPE JUICY SHEET MASK (COSMETIC S...,"mask, cosmetic, sample, testing, skincare, bea...",0.891938,Highest
2,Doldori Premium Gold Essence,"mask, pack, skincare, design, application, pac...",SS0000016020SS APC MASK 13G TUBE,"mask, tube, skin, care, cosmetic, treatment, h...",0.891609,Highest
3,Doldori Premium Gold Essence,"mask, pack, skincare, design, application, pac...",WHITENING MASK 18KG-BAG,"mask, whitening, skin, beauty, treatment, care...",0.883059,Highest
4,Doldori Premium Gold Essence,"mask, pack, skincare, design, application, pac...",(PCPMACAACYQ-I) DERMAFIQUE BIO CELLULOSE TONE ...,"mask, skincare, beauty, treatment, cellulose, ...",0.881926,Highest
...,...,...,...,...,...,...
28005,t:aim Barrier Sunblock 40ml,"sunblock, UV, protection, skin, barrier, ingre...",LID-QUID STRAWBERRY DAIQUIRI 681619816147,"beverage, cocktail, daiquiri, strawberry, liqu...",0.400001,Lowest
28006,t:aim Barrier Sunblock 40ml,"sunblock, UV, protection, skin, barrier, ingre...",COSMETICS POS (POINT OF SALES) MATERIALS SAMPL...,"cosmetics, samples, jars, materials, sales, po...",0.396551,Lowest
28007,t:aim Barrier Sunblock 40ml,"sunblock, UV, protection, skin, barrier, ingre...",GLITTER POWDER (FOR COSMETIC RAW MATERIAL),"glitter, powder, cosmetic, material, beauty, a...",0.391217,Lowest
28008,t:aim Barrier Sunblock 40ml,"sunblock, UV, protection, skin, barrier, ingre...",HINDU RELIGIOUS MATERIALS FOR POOJA PURPOSE-PO...,"pooja, materials, items, religious, hindu, ass...",0.371899,Lowest


In [ ]:
openai_similarity_df_sorted.to_csv('openai_similarity_df_sorted.csv', index=False)